Попрактикуемся с тем, что изучили¶
Будем практиковаться на датасете: https://www.kaggle.com/c/avito-demand-prediction

Ваша задача:

Создать Dataset для загрузки данных (используем только числовые данные)
Обернуть его в Dataloader
Написать архитектуру сети, которая предсказывает число показов на основании числовых данных (вы всегда можете нагенерить дополнительных факторов). Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)
Учить будем на функцию потерь с кагла (log RMSE) - нужно её реализовать
Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели
train-test разделение нужно сделать с помощью sklearn random_state=13, test_size = 0.25

Вопросы? в личку @Kinetikm

In [1]:
import pandas as pd
import numpy as np
import math
import torch
from hw_03 import AvitoData

In [2]:
data = pd.read_csv('data2/tran.csv')

In [3]:
data.head(2)

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000


In [4]:
data = data[['price', 'item_seq_number', 'image_top_1', 'deal_probability']]
data = data.loc[~data['price'].isna()]

class AvitoData записал в отдельны Файл

In [5]:
# class AvitoData(torch.utils.data.Dataset):
#     def __init__(self, data):
#         self.data = data
        
#     def __len__(self):
#         return self.data.shape[0]
    
#     def __getitem__(self, index):
#         return self.data.iloc[index]
        

In [6]:
X = data[['price', 'item_seq_number', 'image_top_1']]
y = data['deal_probability']

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=13, test_size = 0.25)

In [9]:
train = X_train
train['y'] = y_train
test = X_test

<ipython-input-9-8eb665eb7bc4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['y'] = y_train


In [10]:
#train = train.reset_index()


In [25]:
dataset = AvitoData(np.array(train))
train_loader = torch.utils.data.DataLoader(dataset, batch_size=1000, shuffle=True, num_workers=3, drop_last=True)

In [26]:
import torch.nn.functional as F
import torch.nn as nn

In [27]:
class Perceptron(nn.Module):
    def __init__(self, input_dim, output_dim, activation="relu"):
        super(Perceptron, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        self.activation = activation
    
    def forward(self, x):
        x = self.fc(x)
        if self.activation=="relu":
            return F.relu(x)
        if self.activation=="sigmoid":
            return F.sigmoid(x)
        #если ни одна не сработает
        raise RuntimeError

Dropout является методом решения проблемы переобучения, то что обычно происходит в глубоких нейронных сетях, тут же унас маленькая
сетка с небольшим уровнем нелинейности, ей бы хоть какие-то закономерности уловить)))))

In [28]:
class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(Net, self).__init__()
        self.bn1 = nn.BatchNorm1d(input_dim)
        self.fc1 = Perceptron(input_dim, hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        self.fc2 = Perceptron(hidden_dim, 10, "relu")
    
    def forward(self, x):
        x = self.bn1(x)
        x = self.fc1(x)
        x = self.bn2(x)
        x = self.fc2(x)
        return x

In [69]:
net = Net(3, 10)

optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [70]:
from tqdm import tqdm

In [125]:
net = net.float()

In [128]:
for epoch in tqdm(range(10)):  
    running_loss = 0.0
    
    
    for i, data1 in enumerate(train_loader, 0):
        data1 = pd.DataFrame(np.array(data1))
        data1.dropna()
        inputs = torch.tensor(np.array(data1[[0, 1, 2]])).type(torch.LongTensor)
        labels = torch.tensor(np.array(data1[3])).type(torch.LongTensor)
        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 300))
            running_loss = 0.0

  0%|          | 0/10 [00:00<?, ?it/s]

[1,     1] loss: 0.008


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f9682f61d30>
Traceback (most recent call last):
  File "/Users/maximcucer/opt/anaconda3/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1324, in __del__
    self._shutdown_workers()
  File "/Users/maximcucer/opt/anaconda3/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1291, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'


[1,   301] loss: 1.018
[1,   601] loss: 0.356
[1,   901] loss: 0.257


 10%|█         | 1/10 [00:32<04:53, 32.66s/it]

...

 80%|████████  | 8/10 [03:58<00:57, 28.67s/it]

[9,     1] loss: 0.001
[9,   301] loss: 0.174
[9,   601] loss: 0.178
[9,   901] loss: 0.175


 90%|█████████ | 9/10 [04:27<00:28, 28.54s/it]

[10,     1] loss: 0.001
[10,   301] loss: 0.177
[10,   601] loss: 0.175
[10,   901] loss: 0.175


100%|██████████| 10/10 [04:55<00:00, 29.55s/it]


In [130]:
net2 = Net(3, 10)

optimizer = torch.optim.RMSprop(net.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [131]:
net2 = net2.float()

In [133]:
for epoch in tqdm(range(10)):  
    running_loss = 0.0
    
    
    for i, data1 in enumerate(train_loader, 0):
        data1 = pd.DataFrame(np.array(data1))
        data1.dropna()
        inputs = torch.tensor(np.array(data1[[0, 1, 2]])).type(torch.LongTensor)
        labels = torch.tensor(np.array(data1[3])).type(torch.LongTensor)
        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net2(inputs.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 300))
            running_loss = 0.0

  0%|          | 0/10 [00:00<?, ?it/s]

[1,     1] loss: 0.008
[1,   301] loss: 2.367
[1,   601] loss: 2.367
[1,   901] loss: 2.368


 10%|█         | 1/10 [00:42<06:20, 42.32s/it]

[2,     1] loss: 0.008
[2,   301] loss: 2.367
[2,   601] loss: 2.368
[2,   901] loss: 2.367


 20%|██        | 2/10 [01:07<04:57, 37.25s/it]

...

 80%|████████  | 8/10 [03:39<00:52, 26.39s/it]

[9,     1] loss: 0.008
[9,   301] loss: 2.367
[9,   601] loss: 2.368
[9,   901] loss: 2.368


 90%|█████████ | 9/10 [04:04<00:25, 25.86s/it]

[10,     1] loss: 0.008
[10,   301] loss: 2.368
[10,   601] loss: 2.367
[10,   901] loss: 2.368


100%|██████████| 10/10 [04:28<00:00, 26.82s/it]


In [135]:
net3 = Net(3, 10)

optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [136]:
net3 = net3.float()

In [138]:
for epoch in tqdm(range(5)):  
    running_loss = 0.0
    
    
    for i, data1 in enumerate(train_loader, 0):
        data1 = pd.DataFrame(np.array(data1))
        data1.dropna()
        inputs = torch.tensor(np.array(data1[[0, 1, 2]])).type(torch.LongTensor)
        labels = torch.tensor(np.array(data1[3])).type(torch.LongTensor)
        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net3(inputs.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 300))
            running_loss = 0.0

  0%|          | 0/5 [00:00<?, ?it/s]

[1,     1] loss: 0.008
[1,   301] loss: 2.304
[1,   601] loss: 2.304
[1,   901] loss: 2.305


 20%|██        | 1/5 [00:27<01:49, 27.41s/it]

[2,     1] loss: 0.008
[2,   301] loss: 2.305
[2,   601] loss: 2.304
[2,   901] loss: 2.304


 40%|████      | 2/5 [00:55<01:23, 27.71s/it]

...

 60%|██████    | 3/5 [01:27<00:57, 28.85s/it]

[4,     1] loss: 0.008
[4,   301] loss: 2.305
[4,   601] loss: 2.305
[4,   901] loss: 2.305


 80%|████████  | 4/5 [01:55<00:28, 28.64s/it]

[5,     1] loss: 0.008
[5,   301] loss: 2.304
[5,   601] loss: 2.305
[5,   901] loss: 2.305


100%|██████████| 5/5 [02:22<00:00, 28.40s/it]


замечу что всес 3 сетям не нужно 10 эпох, и после ыторой они loss перестает падать, лучше всего сошлась SGD.